In [3]:
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration,GPT2Tokenizer,BertTokenizerFast
import torch
import io
from string import punctuation

import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords


app = FastAPI()
STOP_WORDS = {
    'a', 'an', 'and', 'the', 'is', 'are', 'was', 'were', 'in', 'on', 'of', 'for',
    'with', 'to', 'at', 'from', 'by', 'as', 'that', 'this', 'it', 'be', 'or', 'not',
    'has', 'have', 'had', 'but', 'if', 'they', 'he', 'she', 'we', 'you', 'i'
}

NOUN_TAGS = {'NN', 'NNS', 'NNP', 'NNPS'}
ADJ_TAGS = {'JJ', 'JJR', 'JJS'}
# Load BLIP-2 Model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
blip_processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
blip_model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/NhienLQ/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/NhienLQ/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [16]:
def extract_tags_from_caption(caption):
    words = word_tokenize(caption.lower())
    words = [w for w in words if w not in stopwords and w not in punctuation]
    # pos_tags = nltk.pos_tag(words)

    # tags = [word for word, pos in pos_tags if pos in NOUN_TAGS or pos in ADJ_TAGS]
    return words

def generate_caption(image):
    #image = Image.open(io.BytesIO(image_bytes)).convert('RGB')
    inputs = blip_processor(image, return_tensors="pt").to(device)

    with torch.no_grad():
        output = blip_model.generate(**inputs, max_new_tokens=50)

    caption = blip_processor.decode(output[0], skip_special_tokens=True)

    tags = extract_tags_from_caption(caption)
    return tags

In [17]:
img = Image.open("../pose_server/images/running1.jpg")

In [18]:
tags = generate_caption(img)

TypeError: argument of type 'LazyCorpusLoader' is not iterable

In [12]:
tags

['woman', 'running', 'beach']

In [1]:
nltk.download('punkt_tag')

NameError: name 'nltk' is not defined